In [1]:
import pandas as pd
import numpy as np 

In [2]:
xlsx = pd.ExcelFile('C:/Users/Hajar/Downloads/Alternance Data scientist - Cas 1.xlsx')
xlsx.sheet_names

['Liste de comptes',
 'Company_Info',
 'JobTitle Correspondance',
 'Linkedin job URL',
 'wttj jobs offers']

In [3]:
df1 = pd.read_excel(xlsx, 'Liste de comptes')
df2 = pd.read_excel(xlsx, 'Company_Info')
df3 = pd.read_excel(xlsx, 'JobTitle Correspondance')
df4 = pd.read_excel(xlsx, 'Linkedin job URL')
df5 = pd.read_excel(xlsx, 'wttj jobs offers')

In [4]:
print(df1.columns)
print(df2.columns)
print(df3.columns)
print(df4.columns)
print(df5.columns)

Index(['Linkedin url', 'Company ID', 'Account Name', 'Country'], dtype='object')
Index(['name', 'logo', 'banner', 'description', 'tagLine', 'followerCount',
       'website', 'domain', 'industry', 'companySize', 'headquarters', 'type',
       'founded', 'specialties', 'query', 'mainCompanyID', 'industryCode',
       'salesNavigatorLink', 'employeesOnLinkedIn', 'timestamp', 'companyUrl',
       'companyAddress', 'Comany URL Number 1', 'Comany URL Number 2',
       'Unnamed: 24', 'Unnamed: 25'],
      dtype='object')
Index(['Unnamed: 0', 'Unnamed: 1'], dtype='object')
Index(['url', 'companyName', 'jobTitle', 'JObTitle Correspondance'], dtype='object')
Index(['Company name', 'jobCorrespondance', 'jobDescription', 'jobLink',
       'jobLocation', 'jobRef', 'jobTitle', 'jobType', 'postDate',
       'timestamp'],
      dtype='object')


#### Suppression des doublons

In [5]:
df1=df1.drop_duplicates(subset=['Account Name'], keep='first')
df2=df2.drop_duplicates(subset=['name'], keep='last')

#### Compter les messing values (linkedin url)

In [6]:
df1['Linkedin url'].isnull().sum()

505

In [7]:
df1[df1['Linkedin url'].isna()]

,Linkedin url,Company ID,Account Name,Country
8,NaN,NaN,Cricut,France
9,NaN,NaN,Pimpant,France
17,NaN,NaN,Nalo,France
21,NaN,NaN,Printemps,France
22,NaN,NaN,AVEKAPETI,France
...,...,...,...,...
4792,NaN,NaN,Everli,NaN
4793,NaN,NaN,DoiT International,NaN
4794,NaN,NaN,Superchef,NaN
4795,NaN,NaN,Bigblue,NaN


#### nbre de linkedin urls qu'on n'a pa trouvé

In [8]:
len(set(df1[df1['Linkedin url'].isna()]['Account Name'].unique())- set(df2['name'].unique()))

354

## merger les deux tables : Company ID from mainCompanyID (table 2)

In [9]:
d1=df1[['Account Name','Company ID']]

In [10]:
d2=df2[['name','mainCompanyID']]

In [11]:
df1['Company ID']=d1.merge(d2, left_on='Account Name', right_on='name', how= 'left').drop_duplicates(subset=['Account Name'], keep='first').mainCompanyID

## 2028 of 4790 qu'on n'a pas trouvé leurs Company ID 

In [12]:
c2 = df2[['name','companyUrl']]

In [13]:
c1 = df1[['Linkedin url','Account Name']][df1['Linkedin url'].isna()]

In [ ]:
#df1['Linkedin url'] = df1['Linkedin url'].astype("string")

In [ ]:
#df1[~df1['Linkedin url'].str.startswith('https://www.linkedin.com')]

In [14]:
url = c1.merge(c2, left_on='Account Name', right_on='name', how= 'left').drop_duplicates(subset=['Account Name'], keep='first').companyUrl
df1.loc[df1['Linkedin url'].isna(),'Linkedin url']= url

In [ ]:
#df1['Linkedin url'] = 'https://www.linkedin.com/company/' + df1['Account Name'].str.lower().str.replace(' ', '-')

In [15]:
liste = df1[df1['Linkedin url'].isnull()]['Account Name']

In [16]:
research_later = liste

In [17]:
research_later = research_later.reset_index()['Account Name']

In [18]:
research_later = research_later.astype(str)

In [19]:
import requests
from bs4 import BeautifulSoup
results = []
headers={'User-Agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.103 Safari/537.36'}
for i in research_later : 
    goog_search = "https://www.google.com/search?&q=site:linkedin.com/ AND " + i
    r = requests.get(goog_search,headers=headers).text
    soup = BeautifulSoup(r, "html.parser")
    soup = str(soup)
    if "linkedin.com/company/" in soup :
        extracted = soup.split("linkedin.com/company/")[1].split('&amp;')[0].split('%')[0]
        print(extracted)
        results.append(extracted ) 
        
print(results)

nalo
printemps
ardian
sergic
habx
yessselectrique
alternego
asmodee-us
avico
avis-locataire
citeos/
culture-g
cuure
designerbox
ecotable
forest-style-group-sas
framatome/life
hem
infotel
kusmi-tea
foodandcook
leroy-merlin/life
microids
observia
avospapilles
perlin-network
pierre-fabre
planet-media-group
scptime
sirva-worldwide
western-tidewater-regional-jail-authority
yeswegreen
yupwegoinsurance
ginestet-s.a.
labsterium
the-bradery
schoolab
dalmata-hospitality
leo-pharma-france
temsega
compagnie-des-alpes
cap-fi-technology
eres---groupe-chanel
monoprix-online
colonies
laws6-pandora-jewelry
frenchfounders
adisseo
artify-fr/
exaprint
skema-business-and-law
turbo-fonte-sas
auum
cedrus-partners
red-bull-racing
monnaie-de-paris
naf-naf
orientis-gourmet
cpme-paris-ile-de-france
axteu
syquant-capital
cheerzofficial
mydigitalschool-caen
iceroll
neamedia
test-science/
wefast-it
pharmactiv
attijariwafa-bank-europe
f4-group
lv-technologies-llc
edf
advini
gaminho
banco-santander
paris-group
sharen

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

#### liste des urls using web scraping

In [23]:
results

['nalo',
 'printemps',
 'ardian',
 'sergic',
 'habx',
 'yessselectrique',
 'alternego',
 'asmodee-us',
 'avico',
 'avis-locataire',
 'citeos/',
 'culture-g',
 'cuure',
 'designerbox',
 'ecotable',
 'forest-style-group-sas',
 'framatome/life',
 'hem',
 'infotel',
 'kusmi-tea',
 'foodandcook',
 'leroy-merlin/life',
 'microids',
 'observia',
 'avospapilles',
 'perlin-network',
 'pierre-fabre',
 'planet-media-group',
 'scptime',
 'sirva-worldwide',
 'western-tidewater-regional-jail-authority',
 'yeswegreen',
 'yupwegoinsurance',
 'ginestet-s.a.',
 'labsterium',
 'the-bradery',
 'schoolab',
 'dalmata-hospitality',
 'leo-pharma-france',
 'temsega',
 'compagnie-des-alpes',
 'cap-fi-technology',
 'eres---groupe-chanel',
 'monoprix-online',
 'colonies',
 'laws6-pandora-jewelry',
 'frenchfounders',
 'adisseo',
 'artify-fr/',
 'exaprint',
 'skema-business-and-law',
 'turbo-fonte-sas',
 'auum',
 'cedrus-partners',
 'red-bull-racing',
 'monnaie-de-paris',
 'naf-naf',
 'orientis-gourmet',
 'cpme-par

In [22]:
'linkedin.com/company/' + results

TypeError: can only concatenate str (not "list") to str

In [24]:
 'linkedin.com/company/' + pd.Series(results) 

0                              linkedin.com/company/nalo
1                         linkedin.com/company/printemps
2                            linkedin.com/company/ardian
3                            linkedin.com/company/sergic
4                              linkedin.com/company/habx
                             ...                        
101                       linkedin.com/company/freetrade
102                   linkedin.com/company/jpmorganchase
103                     linkedin.com/company/pitchup-com
104    linkedin.com/company/jet-support-services-inc-...
105    linkedin.com/company/3s-pharmacological-consul...
Length: 106, dtype: object

In [25]:
url_company = 'linkedin.com/company/' + pd.Series(results) 

In [ ]:
df1.loc[df1['Linkedin url'].isnull(),'Linkedin url'] = url_company